In [1]:
import pandas as pd
from os.path import join

In [52]:
data_path = "/media/aviral/8018AF4F18AF4350/Documents and Settings/joavi/Documents/B-TECH/7th SEM/AML/Hackathon/Dataset-1/selfie_dataset.txt"#join("..", "..", "Dataset-1", "selfie_dataset.txt")
image_path = "/media/aviral/8018AF4F18AF4350/Documents and Settings/joavi/Documents/B-TECH/7th SEM/AML/Hackathon/Dataset-1/images"#join("..", "..", "Dataset-1", "selfie_dataset.txt")#join("..", "..", "Dataset-1", "images")

In [62]:
headers = [
    "image_name", "score", "partial_faces" ,"is_female" ,"baby" ,"child" ,"teenager" ,"youth" ,"middle_age" ,"senior" ,"white" ,"black" ,"asian" ,"oval_face" ,"round_face" ,"heart_face" ,"smiling" ,"mouth_open" ,"frowning" ,"wearing_glasses" ,"wearing_sunglasses" ,"wearing_lipstick" ,"tongue_out" ,"duck_face" ,"black_hair" ,"blond_hair" ,"brown_hair" ,"red_hair" ,"curly_hair" ,"straight_hair" ,"braid_hair" ,"showing_cellphone" ,"using_earphone" ,"using_mirror", "braces" ,"wearing_hat" ,"harsh_lighting", "dim_lighting"
]
df_image_details = pd.read_csv(data_path, names=headers, delimiter=" ")
df_image_details.head(3)

,image_name,score,partial_faces,is_female,baby,child,teenager,youth,middle_age,senior,...,curly_hair,straight_hair,braid_hair,showing_cellphone,using_earphone,using_mirror,braces,wearing_hat,harsh_lighting,dim_lighting
0,00a454da495e11e28a7322000a1fa414_6,3.901,1,1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,00cddb96ac4c11e3a30212279ba1b65f_6,4.385,1,1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,01cdd7aa1a1a11e2aaa822000a1fb0dd_6,4.243,-1,1,-1,-1,1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [63]:
df_image_details = df_image_details[df_image_details.is_female != 0]
df_image_details.replace(to_replace=-1, value=0, inplace=True)

In [64]:
image_names = df_image_details.image_name.values.copy()
image_scores = df_image_details[headers[1]].values.copy()
image_attrs = df_image_details[headers[2:]].values.copy()

In [65]:
image_paths = [join(image_path, iname) + '.jpg' for iname in image_names]

In [66]:
from keras.utils import Sequence
import numpy as np
import cv2

In [67]:
class ImageGenerator(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y1 = self.y[0][idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y2 = self.y[1][idx * self.batch_size:(idx + 1) * self.batch_size]

        # read your data here using the batch lists, batch_x and batch_y
        x = [self.read_image(filename) for filename in batch_x] 
        y1 = [atrributes for atrributes in batch_y1]
        y2 = [atrributes for atrributes in batch_y2]
        return np.array(x), [np.array(y1), np.array(y2)]
    
    def read_image(self, fname):
        im = cv2.imread(fname)
        im = cv2.resize(im, (224, 224))
        return im;

# Training Model


In [35]:
from keras.applications import resnet50
from keras.layers import Dense
from keras.models import Model, load_model

In [13]:
model_rnet = load_model("resnet50.hdf5")

Instructions for updating:
keep_dims is deprecated, use keepdims instead


/mnt/Xtension/Ubuntu_Files/conda_virtual_envs/py3.6/lib/python3.6/site-packages/keras/models.py:251: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [14]:
model_rnet.layers.pop()

In [23]:
model_classification = Dense(36, activation='softmax', name='classification')(model_rnet.output)

In [24]:
model_regression = Dense(1, name='regression')(model_rnet.output)

In [25]:
model = Model(inputs=[model_rnet.input], outputs=[model_regression, model_classification])

In [26]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [59]:
model.compile(
    optimizer='adam',
    loss={
        'regression': 'mean_squared_error',
        'classification': 'binary_crossentropy'
    },
    metrics=[
        'accuracy'
    ]
)

In [68]:
train_gen = ImageGenerator(image_paths, (image_scores, image_attrs), batch_size=8)
test_gen = ImageGenerator(image_paths, (image_scores, image_attrs), batch_size=8)

In [69]:
model.fit_generator(train_gen, epochs=200, steps_per_epoch=16)

Epoch 1/200
16/16 [==============================] - 37s 2s/step - loss: 20.0855 - regression_loss: 19.5145 - classification_loss: 0.5710 - regression_acc: 0.0000e+00 - classification_acc: 0.8468
Epoch 2/200
16/16 [==============================] - 13s 822ms/step - loss: 19.7389 - regression_loss: 19.1037 - classification_loss: 0.6351 - regression_acc: 0.0000e+00 - classification_acc: 0.8275
Epoch 3/200
16/16 [==============================] - 13s 821ms/step - loss: 18.5992 - regression_loss: 17.9644 - classification_loss: 0.6348 - regression_acc: 0.0000e+00 - classification_acc: 0.8264
Epoch 4/200
16/16 [==============================] - 13s 821ms/step - loss: 17.6201 - regression_loss: 17.0164 - classification_loss: 0.6038 - regression_acc: 0.0000e+00 - classification_acc: 0.8340
Epoch 5/200
16/16 [==============================] - 13s 822ms/step - loss: 17.6538 - regression_loss: 17.1375 - classification_loss: 0.5163 - regression_acc: 0.0000e+00 - classification_acc: 0.8585
Epoch 6/

16/16 [==============================] - 13s 824ms/step - loss: 10.2994 - regression_loss: 9.8052 - classification_loss: 0.4942 - regression_acc: 0.0000e+00 - classification_acc: 0.8431
Epoch 43/200
16/16 [==============================] - 13s 820ms/step - loss: 9.2367 - regression_loss: 8.7594 - classification_loss: 0.4773 - regression_acc: 0.0000e+00 - classification_acc: 0.8496
Epoch 44/200
16/16 [==============================] - 13s 820ms/step - loss: 10.1693 - regression_loss: 9.6334 - classification_loss: 0.5359 - regression_acc: 0.0000e+00 - classification_acc: 0.8338
Epoch 45/200
 8/16 [==============>...............] - ETA: 6s - loss: 9.1097 - regression_loss: 8.5918 - classification_loss: 0.5179 - regression_acc: 0.0000e+00 - classification_acc: 0.8368

KeyboardInterrupt: 